In [1]:
!pip install git+https://github.com/salaniz/pycocoevalcap.git


  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-5t8407uv
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-5t8407uv
  Resolved https://github.com/salaniz/pycocoevalcap.git to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312245 sha256=d4e3c082772f93fdb603ecc1e7e7c8f51f8fdeb2f58b79c26f4f04dbd2fb6a2f
  Stored in directory: /tmp/pip-ephem-wheel-cache-5mthzao7/wheels/b7/7d/bd/ee763c30e9e11deccc65814eb10c3f58b155fee1d68c0a392f
Successfully built pycocoevalcap


In [12]:
import os
import pandas as pd
import json
import string

# ===== Paths =====
root_csv = "/content/drive/MyDrive/csvs"
generated_files = {
    "Flux-Dev": "meta_captions_Flux-Dev.csv",
    "SDXL": "meta_captions_sdxl.csv",
    "SD2": "meta_captions_sd_2.csv"
}

# Output folder for JSON dictionaries
output_json_folder = os.path.join(root_csv, "cider_dicts")
os.makedirs(output_json_folder, exist_ok=True)

# ===== Text cleaning function =====
def clean_text(text):
    if pd.isna(text) or str(text).strip() == "":
        return None
    text = str(text).lower().strip()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# ===== Function to convert CSV to CIDEr dicts =====
def csv_to_cider_dicts(file_path):
    df = pd.read_csv(file_path)
    df = df.dropna(subset=["Prompts", "Meta Caption"])

    refs = {}
    cands = {}

    for _, row in df.iterrows():
        image_id = str(row["image_name"])
        ref_text = clean_text(row["Prompts"])
        cand_text = clean_text(row["Meta Caption"])
        if ref_text and cand_text:
            refs[image_id] = [ref_text]   # refs must be a list
            cands[image_id] = [cand_text]   # candidate is single string

    return refs, cands

# ===== Process all CSVs and save JSONs =====
for model_name, file_name in generated_files.items():
    file_path = os.path.join(root_csv, file_name)
    refs, cands = csv_to_cider_dicts(file_path)

    refs_json_path = os.path.join(output_json_folder, f"{model_name}_refs.json")
    cands_json_path = os.path.join(output_json_folder, f"{model_name}_cands.json")

    with open(refs_json_path, "w") as f:
        json.dump(refs, f, indent=4)
    with open(cands_json_path, "w") as f:
        json.dump(cands, f, indent=4)

    print(f"Saved CIDEr dictionaries for {model_name}:")
    print(f"  refs -> {refs_json_path}")
    print(f"  cands -> {cands_json_path}")


Saved CIDEr dictionaries for Flux-Dev:
  refs -> /content/drive/MyDrive/csvs/cider_dicts/Flux-Dev_refs.json
  cands -> /content/drive/MyDrive/csvs/cider_dicts/Flux-Dev_cands.json
Saved CIDEr dictionaries for SDXL:
  refs -> /content/drive/MyDrive/csvs/cider_dicts/SDXL_refs.json
  cands -> /content/drive/MyDrive/csvs/cider_dicts/SDXL_cands.json
Saved CIDEr dictionaries for SD2:
  refs -> /content/drive/MyDrive/csvs/cider_dicts/SD2_refs.json
  cands -> /content/drive/MyDrive/csvs/cider_dicts/SD2_cands.json


In [14]:
import os
import json
from pycocoevalcap.cider.cider import Cider

# ===== Paths =====
root_csv = "/content/drive/MyDrive/csvs"
cider_json_folder = os.path.join(root_csv, "cider_dicts")
output_json = os.path.join(root_csv, "cider_average_scores.json")

# Models
models = ["Flux-Dev", "SDXL", "SD2"]

# ===== Compute CIDEr =====
average_scores = {}

for model_name in models:
    refs_path = os.path.join(cider_json_folder, f"{model_name}_refs.json")
    cands_path = os.path.join(cider_json_folder, f"{model_name}_cands.json")

    # Load JSON dictionaries
    with open(refs_path) as f:
        refs = json.load(f)
    with open(cands_path) as f:
        cands = json.load(f)

    try:
        scorer = Cider()
        score, _ = scorer.compute_score(refs, cands)
        average_scores[model_name] = score
        print(f"CIDEr score for {model_name}: {score}")
    except Exception as e:
        print(f"Error computing CIDEr for {model_name}: {e}")
        average_scores[model_name] = None

# ===== Save averages to JSON =====
with open(output_json, "w") as f:
    json.dump(average_scores, f, indent=4)

print("Average CIDEr scores saved to:", output_json)
print(average_scores)


CIDEr score for Flux-Dev: 0.037878950842012274
CIDEr score for SDXL: 0.024955449323365252
CIDEr score for SD2: 0.02940295592983221
Average CIDEr scores saved to: /content/drive/MyDrive/csvs/cider_average_scores.json
{'Flux-Dev': np.float64(0.037878950842012274), 'SDXL': np.float64(0.024955449323365252), 'SD2': np.float64(0.02940295592983221)}
